In [9]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
import numpy as np
from src import parse
from collections import Counter

/Users/andrew/src/otp-scheduler


In [10]:
def sort_schedule(df):
    return df.sort_values(by=['zone', 'priority_rank', 'code']).reset_index(drop=True)

restaurants = parse.parse_from_restaurants_csv('../../examples/boston/20200415/restaurants.csv').set_index('code')
hospitals = sort_schedule(parse.parse_from_hospital_requests_csv('../../examples/boston/20200415/hospitals.csv')).set_index('name')
vip_hospitals = sort_schedule(parse.parse_from_hospital_requests_csv('../../examples/boston/20200415/vips.csv')).set_index('name')
vip_schedule = vip_hospitals.iloc[:, 4:]

# Create empty results frame
output = hospitals.copy()
output.iloc[:, 4:] = np.nan

rest_usage = restaurants.copy()
rest_usage.iloc[:, 2:] = 0

rest_cap = restaurants.copy()

# ASSUMPTIONS
MEAL_CAPACITY = 20000
MAX_DROPOFFS_PER_RUN = 4
MIN_MEALS_PER_RUN = 50

curr_capacity = MEAL_CAPACITY

total_hospitals = hospitals.shape[0]

### Idea #1

1. Go in order of hospital priority + who has least meals served so far (start with VIPs?)
2. find the order request that’s the most meals (or a request that’s in the biggest unserved cluster) that hasn’t yet been fulfilled. 
3. Assign a restaurant that can handle that restaurant and hopefully everything else in the cluster. 
4. Adjust meal counts for other hospitals that will be delivered to in that cluster. 
5. Update meals for all hospitals, capacity for that restaurant, and repeat. 

In [11]:
def day_from_meal(meal):
    return meal.replace('lunch', '').replace('breakfast', '').replace('dinner', '').replace('_', '')

def get_meal_log(schedule, is_vip):
    log = []
    for name, row in schedule.iterrows():
        for j, val in enumerate(row[4:]):
            if not pd.isnull(val):
                log.append([
                    name,
                    row['zone'],
                    row['priority_rank'],
                    is_vip,
                    schedule.columns[j + 4],
                    val, 
                    np.nan,
                    day_from_meal(schedule.columns[j + 4]),
                    False
                ])
    return pd.DataFrame(log, columns=['hospital', 'zone', 'priority_rank', 'vip', 'meal', 'quantity', 'restaurant', 'day', 'failed'])

vip_log = get_meal_log(vip_hospitals, True)
meal_log = get_meal_log(hospitals, False).append(vip_log)

In [12]:
# meal_log

In [13]:
# 1. Fill outstanding VIP meals
# 2. Fill out with LCD approach (select min, if tie take one with better priority)

In [14]:
def maybe_compact(df, meal, compact):
    if compact:
        return df[df[meal].notnull()]
    return df

def get_rest_cap_cluster_for_meal(meal, compact=False):
    r = rest_cap[[meal]]
    return maybe_compact(r, meal, compact)

In [15]:
def get_oustanding_vip_meals():
    return meal_log[(meal_log['vip'] == True) & meal_log['restaurant'].isnull()]

def get_outstanding_hospitals_in_cluster_for_meal(meal, zone):
    return meal_log[(meal_log['meal'] == meal) & (meal_log['zone'] == zone) & meal_log['restaurant'].isnull()]

def get_hospitals_in_cluster_for_meal(meal, zone):
    return meal_log[(meal_log['meal'] == meal) & (meal_log['zone'] == zone)]

def is_restaurant_serving_same_day_other_meal(rest_code, meal):
    day = day_from_meal(meal)
    rest_serving = meal_log[(meal_log['restaurant'] == rest_code) & (meal_log['day'] == day) & meal_log['meal'] != meal]
    return not rest_serving.empty

def assign_hospital_order(row):
    global curr_capacity

    meal = row['meal']
    hospital = row['hospital']
    
    print('Assigning', hospital, meal)
    
    # Check to see if we've already done this (sanity)
    if not pd.isnull(row['restaurant']):
        print('We have already assigned this one!')
        return False
        
    hospitals_in_cluster = get_hospitals_in_cluster_for_meal(meal, row['zone'])

    # Restaurant capacity
    r_cap = get_rest_cap_cluster_for_meal(meal)
        
    # The amount have to serve
    target_quantity = row['quantity']
    
    if curr_capacity - target_quantity < 0:
        print('We cannot to afford this meal because of our global limit!', curr_capacity, target_quantity)
        return False
    
    print(hospital, meal, row['zone'], target_quantity)
        
#     def get_candidate():
#         # Try to serve other (bonus) hospitals alongside this hospital, up to the MAX_DROPOFFS_PER_RUN limit
#         other_count = min(MAX_DROPOFFS_PER_RUN - 1, others_in_cluster.shape[0])
#         while(other_count >= 0):
#             # Sum of the remaining hospitals quantity, up to MAX_DROPOFFS_PER_RUN - 1, SORTED BY HIGHEST QUANTITY
#             other_quantity = others_in_cluster.iloc[:other_count, :]['quantity'].sum()

#             # Try to serve these on the same run
#             bonus_quantity = target_quantity + other_quantity
            
#             if bonus_quantity + target_quantity > curr_capacity:
#                 other_count -= 1
#                 continue

#             # Find any restaurants that can serve the hospital + bonus
#             r_candidates = r_cap[r_cap[meal] >= bonus_quantity]
            
#             # Take the one that's closest to the bonus
#             r_candidates = r_candidates[r_candidates[meal] == r_candidates[meal].min()]

#             # Found one!
#             if not r_candidates.empty:
#                 return r_candidates, other_count
            
#             # Otherwise try again, with one fewer bonus restaurant
#             other_count -= 1
        
#         return None, None
    
    def find_restaurant():
        # Check if a restaurant is already serving the cluster
        restaurants_serving_cluster = hospitals_in_cluster[hospitals_in_cluster['restaurant'].notnull()]['restaurant']
        
        if not restaurants_serving_cluster.empty:
            # Filter out any that are already at the dropout limit
            rest_counter = Counter(restaurants_serving_cluster)
            rest_cluster_keys = rest_counter.keys()
            print('Checking for restaurants already in cluster', rest_counter)
            rest_with_room = [c[0] for c in rest_counter.most_common() if c[1] < MAX_DROPOFFS_PER_RUN]
            
            # Find restaurants with capacity
            candidates_with_quantity = []
            for r in rest_with_room:
                q = rest_cap.loc[r, meal]
                if q >= target_quantity:
                    print('Found an existing restaurant with capacity', r, q)
                    candidates_with_quantity.append(r)
                    if is_restaurant_serving_same_day_other_meal(r, meal):
                        print('Returning a restaurant with capacity, working on the same day', r, q)
                        return r
            if len(candidates_with_quantity):
                print('Returning a restaurant with capacity', r, q)
                return candidates_with_quantity[0]
        
        print('No suitable restaurants already working this cluster')
        # Couldn't find an appropriate restaurant already serving cluster
        
        restaurants_in_other_zones = meal_log[(meal_log['meal'] == meal) & (meal_log['zone'] != row['zone']) & (meal_log['restaurant'].notnull())]['restaurant'].unique()
        
        print('filtering our candidates with two lists', restaurants_serving_cluster, restaurants_in_other_zones)
        r_candidates = r_cap[(~r_cap.index.isin(restaurants_serving_cluster)) & (~r_cap.index.isin(restaurants_in_other_zones)) & (r_cap[meal] >= target_quantity)]
        
        print('Number of candidates with capacity', r_candidates.shape[0])
        
        if r_candidates.empty:
            return None
        
        # Sort in quantity order
        r_candidates = r_candidates.sort_values(by=meal, ascending=False)
        
        day = day_from_meal(meal)
        
        # TODO Make sure restaurant isn't overbooked on their max day count
        for i, r in r_candidates.iterrows():
            print(i)
            days_worked = set(meal_log[meal_log['restaurant'] == i]['day'])
            # Check if we're already working this day, or the new day is still under cap
            if day in days_worked or len(days_worked) < rest_cap.loc[i, 'max_days_per_week']:
                return i
        
        return None
          
    r_code = find_restaurant()
    
    if r_code is None:
        print('CANNOT SERVE HOSPITAL FOR THEIR REQUEST', hospital, meal, r_cap)
        return False
                
    # Allocate the restaurant to the hospital(s)
    print('debiting r capacity: ', r_code, rest_cap.loc[r_code, meal])
    rest_cap.loc[r_code, meal] -= target_quantity
            
    print('increasing r usage', r_code, target_quantity)
    rest_usage.loc[r_code, meal] += target_quantity
        
    curr_capacity -= target_quantity
                
    print('updating log', hospital, meal, r_code)
    meal_log.loc[(meal_log['hospital'] == hospital) & (meal_log['meal'] == meal), 'restaurant'] = r_code
    
    return True
    
#     print('updating remaining assignments', other_count)
#     other_amt = 0
#     for idx in range(other_count):
#         h = others_in_cluster.iloc[idx]
#         print('\t', h['hospital'], h['quantity'])
#         rest_usage.loc[r_code, meal] += h['quantity']
#         curr_capacity -= h['quantity']
#         other_amt += h['quantity']
#         meal_log.loc[(meal_log['hospital'] == h['hospital']) & (meal_log['meal'] == meal), 'restaurant'] = r_code
#     print('other amount assigned', other_amt)
#     print('total', other_amt + target_quantity)
    
for i, row in get_oustanding_vip_meals().iterrows():
    assigned = assign_hospital_order(row)
    if not assigned:
        meal_log.loc[row['hospital'], 'failed'] = True
    print()
    print()
    

Assigning Brigham and Women's Hospital - ED lunch_wed
Brigham and Women's Hospital - ED lunch_wed 1 100.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: float64) []
Number of candidates with capacity 10
PG
debiting r capacity:  PG 700.0
increasing r usage PG 100.0
updating log Brigham and Women's Hospital - ED lunch_wed PG


Assigning Brigham and Women's Hospital - ED lunch_sun
Brigham and Women's Hospital - ED lunch_sun 1 100.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) []
Number of candidates with capacity 7
PG
debiting r capacity:  PG 700.0
increasing r usage PG 100.0
updating log Brigham and Women's Hospital - ED lunch_sun PG


Assigning Brigham and Women's Hospital - ED dinner_wed
Brigham and Women's Hospital - ED dinner_wed 1 100.0
No suitable restaurants already working this cluster
filtering our candidate

Returning a restaurant with capacity, working on the same day PG 688.0
debiting r capacity:  PG 688.0
increasing r usage PG 150.0
updating log Boston Medical Center dinner_sat PG


Assigning Boston Medical Center dinner_sun
Boston Medical Center dinner_sun 2 150.0
Checking for restaurants already in cluster Counter({'LD': 4})
No suitable restaurants already working this cluster
filtering our candidates with two lists 63    LD
84    LD
7     LD
21    LD
Name: restaurant, dtype: object ['PG']
Number of candidates with capacity 4
SU
debiting r capacity:  SU 400.0
increasing r usage SU 150.0
updating log Boston Medical Center dinner_sun SU


Assigning Harvard Street Neighborhood Health Center lunch_mon
Harvard Street Neighborhood Health Center lunch_mon 2 30.0
Checking for restaurants already in cluster Counter({'PG': 4})
No suitable restaurants already working this cluster
filtering our candidates with two lists 71    PG
98    PG
8     PG
22    PG
Name: restaurant, dtype: object []
Number

In [16]:
# meal_log[meal_log.vip == True]

In [17]:
hospitals_that_failed_placement = set()

def select_and_assign_hospital():
    next_hospital = meal_log[~meal_log.hospital.isin(list(hospitals_that_failed_placement))].groupby('hospital')[['priority_rank', 'restaurant']].agg({
            'restaurant': 'count',
            'priority_rank': 'min'
        }).sort_values(by=['restaurant', 'priority_rank']).reset_index().iloc[0]
    
    print('Next hospital:', next_hospital)

    # Find potential orders to serve, ordered by quantity
    possible_orders = meal_log[(meal_log['hospital'] == next_hospital['hospital']) & (meal_log['restaurant'].isnull()) & (meal_log['failed'] == False)].sort_values(by='quantity', ascending=False)
    
    print('Possible orders', possible_orders)
    for i, row in possible_orders.iterrows():
        if assign_hospital_order(row):
            return True
        else:
            meal_log.loc[
                (meal_log['hospital'] == row['hospital']) & (meal_log['meal'] == row['meal']), 
                'failed'
            ] = True
            
    hospitals_that_failed_placement.add(next_hospital['hospital'])
    

while curr_capacity > 0 and len(hospitals_that_failed_placement) < total_hospitals:
    select_and_assign_hospital()
    print()
    print()

Next hospital: hospital         BIDMC - East Campus
restaurant                         0
priority_rank                      1
Name: 0, dtype: object
Possible orders                hospital  zone  priority_rank    vip         meal  quantity  \
7   BIDMC - East Campus     1              1  False    lunch_mon      30.0   
8   BIDMC - East Campus     1              1  False   lunch_tues      30.0   
9   BIDMC - East Campus     1              1  False    lunch_wed      30.0   
10  BIDMC - East Campus     1              1  False  lunch_thurs      30.0   
11  BIDMC - East Campus     1              1  False    lunch_fri      30.0   
12  BIDMC - East Campus     1              1  False    lunch_sat       6.0   
13  BIDMC - East Campus     1              1  False    lunch_sun       6.0   

   restaurant    day  failed  
7         NaN    mon   False  
8         NaN   tues   False  
9         NaN    wed   False  
10        NaN  thurs   False  
11        NaN    fri   False  
12        NaN    sat   F

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_value

                                            hospital  zone  priority_rank  \
174  Cambridge Health Alliance - Cambridge Hospital     3              1   
175  Cambridge Health Alliance - Cambridge Hospital     3              1   
176  Cambridge Health Alliance - Cambridge Hospital     3              1   
177  Cambridge Health Alliance - Cambridge Hospital     3              1   
178  Cambridge Health Alliance - Cambridge Hospital     3              1   
179  Cambridge Health Alliance - Cambridge Hospital     3              1   
180  Cambridge Health Alliance - Cambridge Hospital     3              1   

       vip         meal  quantity restaurant    day  failed  
174  False    lunch_mon     115.0        NaN    mon   False  
175  False   lunch_tues     115.0        NaN   tues   False  
176  False    lunch_wed     115.0        NaN    wed   False  
177  False  lunch_thurs     115.0        NaN  thurs   False  
178  False    lunch_fri     115.0        NaN    fri   False  
179  False    lunc

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


 lunch_tues 3 500.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) ['PG' 'LD']
Number of candidates with capacity 0
CANNOT SERVE HOSPITAL FOR THEIR REQUEST Massachusetts General Hospital - Central lunch_tues       lunch_tues
code            
PG         288.0
LD         670.0
MM           NaN
PO         300.0
FC         400.0
CS           NaN
SU         400.0
IG         200.0
MD         400.0
SH         300.0
BB         400.0
ME         100.0
Assigning Massachusetts General Hospital - Central lunch_thurs
Massachusetts General Hospital - Central lunch_thurs 3 500.0
Checking for restaurants already in cluster Counter({'FC': 2})
No suitable restaurants already working this cluster
filtering our candidates with two lists 210    FC
39     FC
Name: restaurant, dtype: object ['PG' 'LD']
Number of candidates with capacity 0
CANNOT SERVE HOSPITAL FOR THEIR REQUEST Massachusetts General Hospital - Central lu

...

Returning a restaurant with capacity, working on the same day SU 303.0
debiting r capacity:  SU 303.0
increasing r usage SU 18.0
updating log BWH - Faulkner 7N Covid Unit lunch_sun SU


Next hospital: hospital         BWH - Faulkner ICU
restaurant                       13
priority_rank                     1
Name: 0, dtype: object
Possible orders                hospital  zone  priority_rank    vip        meal  quantity  \
139  BWH - Faulkner ICU     2              1  False  dinner_sun      15.0   

    restaurant  day  failed  
139        NaN  sun   False  
Assigning BWH - Faulkner ICU dinner_sun
BWH - Faulkner ICU dinner_sun 2 15.0
Checking for restaurants already in cluster Counter({'LD': 4, 'SU': 4})
No suitable restaurants already working this cluster
filtering our candidates with two lists 63     LD
84     LD
91     SU
111    SU
125    SU
7      LD
21     LD
28     SU
Name: restaurant, dtype: object ['PG' 'MD']
Number of candidates with capacity 3
BB
debiting r capacity:  BB 400.0


/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_value

 hospital         Newton Wellesley Hospital
restaurant                              14
priority_rank                            1
Name: 0, dtype: object
Possible orders Empty DataFrame
Columns: [hospital, zone, priority_rank, vip, meal, quantity, restaurant, day, failed]
Index: []


Next hospital: hospital         Boston Medical Center
restaurant                          15
priority_rank                        1
Name: 0, dtype: object
Possible orders                   hospital  zone  priority_rank    vip       meal  quantity  \
104  Boston Medical Center     2              1  False  lunch_sun     150.0   

    restaurant  day  failed  
104        NaN  sun   False  
Assigning Boston Medical Center lunch_sun
Boston Medical Center lunch_sun 2 150.0
Checking for restaurants already in cluster Counter({'LD': 4, 'SU': 3})
Found an existing restaurant with capacity SU 285.0
Returning a restaurant with capacity, working on the same day SU 285.0
debiting r capacity:  SU 285.0
increasing r usage

Boston Healthcare for the Homeless breakfast_fri 2 12.0
Checking for restaurants already in cluster Counter({'SH': 1})
Found an existing restaurant with capacity SH 230.0
Returning a restaurant with capacity, working on the same day SH 230.0
debiting r capacity:  SH 230.0
increasing r usage SH 12.0
updating log Boston Healthcare for the Homeless breakfast_fri SH


Next hospital: hospital         Boston Healthcare for the Homeless
restaurant                                       33
priority_rank                                     1
Name: 0, dtype: object
Possible orders                               hospital  zone  priority_rank    vip  \
69  Boston Healthcare for the Homeless     2              1  False   
70  Boston Healthcare for the Homeless     2              1  False   

             meal  quantity restaurant  day  failed  
69  breakfast_sat      12.0        NaN  sat   False  
70  breakfast_sun      12.0        NaN  sun   False  
Assigning Boston Healthcare for the Homeless break

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [18]:
curr_capacity, hospitals_that_failed_placement

(2128.0,
 {'BIDMC - Chelsea',
  'BIDMC - East Campus',
  'BIDMC - ICU East',
  'BIDMC - ICU West',
  'BIDMC - West Campus',
  'BWH - Faulkner 7N Covid Unit',
  'BWH - Faulkner ICU',
  "Boston Children's Hospital - ED",
  'Boston Healthcare for the Homeless',
  'Boston Medical Center',
  "Brigham & Women's Hospital (COVID ICUs)",
  "Brigham and Women's - Faulkner ED",
  "Brigham and Women's Hospital - ED",
  'Cambridge Health Alliance - Cambridge Hospital',
  'Cambridge Health Alliance - Everett Hospital',
  'Cambridge Health Alliance - Somerville Hospital',
  'Carney Hospital - ED',
  'Charles River Community Health',
  'Codman Square Health Center',
  'Dana Farber Cancer Institute - Inpatient PAs',
  'Dot House Health Dorchester',
  'East Boston Neighborhood Health Center',
  'Harvard Street Neighborhood Health Center',
  'Massachusetts General Hospital - Central',
  'Massachusetts General Hospital - PICU',
  'Mattapan Community Health Center',
  'Newton Wellesley Hospital',
  'North 

In [19]:
meal_log[meal_log['restaurant'].notnull()]['quantity'].sum()

21330.0

In [20]:
for i, row in meal_log.iterrows():
    if not pd.isnull(row['restaurant']):
        val = str(row['restaurant'] + ' - ' + str(row['quantity']))
    else:
        val = str('XXX - ' + str(row['quantity']))
    output.loc[row['hospital'], row['meal']] = val
    
# for i, row in meal_log.iterrows():
#     if not pd.isnull(row['restaurant']):
#         output.loc[row['hospital'], row['meal']] = row['restaurant']
        
output.to_csv('../../output.csv')

In [22]:
rest_usage.iloc[:, 2:].sum(axis=1).sort_values(ascending=False)

code
PG    3523.0
SU    2873.0
LD    2650.0
MD    2095.0
BB    1980.0
SH    1739.0
FC    1630.0
PO     995.0
IG     299.0
ME      88.0
CS       0.0
MM       0.0
dtype: float64

In [23]:
meal_log.groupby(['priority_rank', 'hospital']).count()

zone  vip  \
priority_rank hospital                                                        
1             BIDMC - East Campus                                    7    7   
              BIDMC - ICU East                                       2    2   
              BIDMC - ICU West                                       2    2   
              BIDMC - West Campus                                    7    7   
              BWH - Faulkner 7N Covid Unit                          14   14   
              BWH - Faulkner ICU                                    14   14   
              Boston Children's Hospital - ED                        5    5   
              Boston Healthcare for the Homeless                    35   35   
              Boston Medical Center                                 21   21   
              Brigham & Women's Hospital (COVID ICUs)               21   21   
              Brigham and Women's - Faulkner ED                      8    8   
              Brigham and Women's Hospital - ED                      8    8   
              Cambridge Health Alliance - Cambridge Hospital         7    7   
              Cambridge Health Alliance - Everett Hospital           7    7   
              Cambridge Health Alliance - Somerville Hospital        7    7   
              Carney Hospital - ED                                   7    7   
              Dana Farber Cancer Institute - Inpatient PAs           7    7   
              Massachusetts General Hospital - Central               7    7   
              Massachusetts General Hospital - PICU                 20   20   
              Newton Wellesley Hospital                             14   14   
              Tufts Medical Center                                  10   10   
              VA Medical Center - West Roxbury                       6    6   
2             BIDMC - Chelsea                                        5    5   
              Charles River Community Health                         5    5   
              Codman Square Health Center                            5    5   
              Dot House Health Dorchester                            5    5   
              East Boston Neighborhood Health Center                 6    6   
              Harvard Street Neighborhood Health Center             10   10   
              Mattapan Community Health Center                       3    3   
              North End Waterfront Health Center                     4    4   
              South Boston Community Health Center                   5    5   
              Southern Jamaica Plain Health Center (part of B...     5    5   
              Upham's Corner Health Center                           6    6   
              Whittier Street Health Center                          5    5   

                                                                  meal  \
priority_rank hospital                                                   
1             BIDMC - East Campus                                    7   
              BIDMC - ICU East                                       2   
              BIDMC - ICU West                                       2   
              BIDMC - West Campus                                    7   
              BWH - Faulkner 7N Covid Unit                          14   
              BWH - Faulkner ICU                                    14   
              Boston Children's Hospital - ED                        5   
              Boston Healthcare for the Homeless                    35   
              Boston Medical Center                                 21   
              Brigham & Women's Hospital (COVID ICUs)               21   
              Brigham and Women's - Faulkner ED                      8   
              Brigham and Women's Hospital - ED                      8   
              Cambridge Health Alliance - Cambridge Hospital         7   
              Cambridge Health Alliance - Everett Hospital           7   
              Cambridge Health Allian